In [1]:
input_shape=[1, 3, 32, 32]

In [2]:
a = input_shape[1]

In [3]:
a

3

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, **kwargs):
        super(CNN, self).__init__()

        self.in_shape = kwargs['input_shape']
        self.in_channel = kwargs['input_shape'][1]
        self.n_classes = kwargs['n_classes']

        # Convolutional layers
        self.conv1 = nn.Conv2d(self.in_channel, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        self.fc1 = nn.Linear(self._n_fc_units(), 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, self.n_classes)

    def forward(self, x):
        x = self._forward_conv(x)
        x = self._forward_fc(x)
        return F.log_softmax(self.fc3(x), dim=1)

    def _forward_conv(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.relu(self.conv4(x))
        x = self.pool(x)
        return x

    def _forward_fc(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return x

    def _n_fc_units(self):
        dummy_input = torch.rand(self.in_shape)
        return self._forward_conv(dummy_input).view(1, -1).shape[1]

# Instantiate the model
model = CNN(input_shape=(1, 3, 32, 32), n_classes=10)

# Print the model
print(model)


CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=10, bias=True)
)
